## Basic layers
### Convolutional layer
### Pooling layer
### Activation layer

## Convolutional layer

### ```tf.nn``` implementation

In [2]:
import tensorflow as tf

In [42]:
tf.reset_default_graph()
image = tf.random_normal([1,112,96,3])
in_channels = 3
out_channels = 32
kernel_size = 5
#[filter_height,filter_width,in_channels,out_channels]
conv_weight = tf.Variable(tf.truncated_normal([kernel_size,kernel_size,in_channels,out_channels],
                          stddev=0.1,dtype=tf.float32))

bias = tf.Variable(tf.zeros([out_channels],dtype=tf.float32))
#data_format='NHWC'
#input_data with shape [batch,in_height,in_width,in_channels]
#output_data with shape [batch,out_height,out_width,out_channels]
conv = tf.nn.conv2d(image,conv_weight,strides=[1,2,2,1],padding='SAME')
conv = tf.nn.bias_add(conv,bias)


### ```tf.layers``` implementation

In [34]:
tf.reset_default_graph()
image = tf.random_normal([1,112,96,3])
in_channels = 3
out_channels = 32
kernel_size = 5
truncated_norm_init = tf.truncated_normal_initializer(
                          stddev=0.1,dtype=tf.float32)


zero_init = tf.zeros_initializer(dtype=tf.float32)

l2_regularizer= tf.contrib.layers.l2_regularizer(1.0)
conv = tf.layers.conv2d(image,out_channels,[kernel_size,kernel_size],strides=[2,2],padding='SAME',
                 kernel_initializer=truncated_norm_init,bias_initializer=zero_init,
                        kernel_regularizer=l2_regularizer,bias_regularizer=l2_regularizer)


### [The difference between 'SAME' and 'VALID' in tensorflow](https://stackoverflow.com/questions/37674306/what-is-the-difference-between-same-and-valid-padding-in-tf-nn-max-pool-of-t)

- For the SAME padding, the output height and width are computed as:
out_height = ceil(float(in_height) / float(strides[1]))

out_width = ceil(float(in_width) / float(strides[2]))

And

- For the VALID padding, the output height and width are computed as:
out_height = ceil(float(in_height - filter_height + 1) / float(strides[1]))

out_width = ceil(float(in_width - filter_width + 1) / float(strides[2]))


In [56]:
tf.reset_default_graph()
image = tf.random_normal([1,112,96,3])
in_channels = 3
out_channels = 32
kernel_size = 5
conv_weight = tf.Variable(tf.truncated_normal([kernel_size,kernel_size,in_channels,out_channels],
                          stddev=0.1,dtype=tf.float32))
print 'image shape',image.get_shape()
print 'conv weight shape',conv_weight.get_shape()
bias = tf.Variable(tf.zeros([out_channels],dtype=tf.float32))
conv = tf.nn.conv2d(image,conv_weight,strides=[1,2,2,1],padding='SAME')
conv = tf.nn.bias_add(conv,bias)
print 'conv output shape with SAME padded',conv.get_shape()
conv = tf.nn.conv2d(image,conv_weight,strides=[1,2,2,1],padding='VALID')
conv = tf.nn.bias_add(conv,bias)
print 'conv output shape with VALID padded',conv.get_shape()

image shape (1, 112, 96, 3)
conv weight shape (5, 5, 3, 32)
conv output shape with SAME padded (1, 56, 48, 32)
conv output shape with VALID padded (1, 54, 46, 32)


## Pooling layer

In [57]:
pool_size = 3
pool = tf.nn.max_pool(conv,ksize=[1,pool_size,pool_size,1],strides=[1,2,2,1],padding='SAME')
print pool.get_shape()
pool = tf.nn.avg_pool(conv,ksize=[1,pool_size,pool_size,1],strides=[1,2,2,1],padding='SAME')
print pool.get_shape()

(1, 27, 23, 32)
(1, 27, 23, 32)


## Activation layer¶

### tf.nn.relu

In [58]:
relu = tf.nn.relu(pool)

### prelu
```
y = prelu(x) 
if x > 0: 
    y = x
else:
    y = alpha*x #alpha >0
```

In [59]:
def prelu(x, name = 'prelu'):
    with tf.variable_scope(name):
        alphas = tf.get_variable('alpha', x.get_shape()[-1], initializer=tf.constant_initializer(0.25), regularizer = l2_regularizer, dtype = tf.float32)
    pos = tf.nn.relu(x)
    neg = tf.multiply(alphas,(x - abs(x)) * 0.5)
    return pos + neg
prelu_out = prelu(pool)
print prelu_out.get_shape()

(1, 27, 23, 32)
